In [ ]:
# Análise exploratória olhando localização, no applicants, tipo workplace, required skills e level, company_sector

# NLP - modelo de tópicos sobre o campo description - bertopic

In [4]:
import os
import pandas as pd

In [16]:
df_jobs = pd.DataFrame()
for job in os.listdir('data'):
    if job[-4:] == '.csv':
        new_df = pd.read_csv(f'data/{job}', sep=';')
        df_jobs = pd.concat([df_jobs, new_df], axis=0)

In [17]:
df_jobs.shape

(4486, 17)

In [18]:
cols_duplication = ['company_name', 'position', 'location']

len(df_jobs[df_jobs[cols_duplication].duplicated()])

0      False
1      False
2      False
3      False
4      False
       ...  
933    False
934    False
935    False
936    False
937    False
Length: 4486, dtype: bool

In [ ]:
df_jobs['required_skills'].dropna().map(lambda x: eval(x)).explode().value_counts()